# Open Food Facts: the carbon “food-print” we do not eat

## Abstract
<i>Everything we do has a carbon footprint, and our diet is no exception. From growing, farming, processing and packaging our food, energy and organic resources are consumed and released, which reflects in the emission of greenhouse gases, like CO<sub>2</sub>. In our project, we analyze the processed foods industry - its manufacturing, product composition, and sales - for the main sources of carbon emissions, using the Open Food Facts dataset. We explain the carbon footprint repartition, starting on an understanding of the products, followed by the breakdown of production countries as well as point of sales and evaluating trends in diet composition, with a special focus on nutritionally high marked products in France and the UK. 

With this study, we want to provide a better understanding of the agri-food industry, and eventually help reducing carbon emissions.</i>

In this notebook, we are performingt the above analysis on the OpenFoodFacts database, which we pre-processed using the __Open Food Facts - Cleanse Data__ notebook in the main directory.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Abstract" data-toc-modified-id="Abstract-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Abstract</a></span></li><li><span><a href="#Import-cleansed-data" data-toc-modified-id="Import-cleansed-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import cleansed data</a></span></li><li><span><a href="#Analyse-data" data-toc-modified-id="Analyse-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analyse data</a></span><ul class="toc-item"><li><span><a href="#Production-/-manufacture-impact" data-toc-modified-id="Production-/-manufacture-impact-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Production / manufacture impact</a></span><ul class="toc-item"><li><span><a href="#Global-distribution-of-global-food-producers" data-toc-modified-id="Global-distribution-of-global-food-producers-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Global distribution of global food producers</a></span><ul class="toc-item"><li><span><a href="#Which-are-the-dominant-global-food-producers-and-manufacturers?" data-toc-modified-id="Which-are-the-dominant-global-food-producers-and-manufacturers?-3.1.1.1"><span class="toc-item-num">3.1.1.1&nbsp;&nbsp;</span>Which are the dominant global food producers and manufacturers?</a></span></li><li><span><a href="#How-is-this-distribution-impacted-when-we-consider-neutral-and-large-carbon-footprint-products?" data-toc-modified-id="How-is-this-distribution-impacted-when-we-consider-neutral-and-large-carbon-footprint-products?-3.1.1.2"><span class="toc-item-num">3.1.1.2&nbsp;&nbsp;</span>How is this distribution impacted when we consider neutral and large carbon footprint products?</a></span></li></ul></li><li><span><a href="#Case-study:-Palm-oil" data-toc-modified-id="Case-study:-Palm-oil-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Case study: Palm oil</a></span><ul class="toc-item"><li><span><a href="#Can-we-observe-any-trend-in-the-number-of-products-including-this-oil-(assuming-a-strong-dependence-between-date-the-product-was-added-to-the-database-and-data-the-product-was-invented)?" data-toc-modified-id="Can-we-observe-any-trend-in-the-number-of-products-including-this-oil-(assuming-a-strong-dependence-between-date-the-product-was-added-to-the-database-and-data-the-product-was-invented)?-3.1.2.1"><span class="toc-item-num">3.1.2.1&nbsp;&nbsp;</span>Can we observe any trend in the number of products including this oil (assuming a strong dependence between date the product was added to the database and data the product was invented)?</a></span></li><li><span><a href="#Which-country-use-palm-oils-for-production?" data-toc-modified-id="Which-country-use-palm-oils-for-production?-3.1.2.2"><span class="toc-item-num">3.1.2.2&nbsp;&nbsp;</span>Which country use palm oils for production?</a></span></li></ul></li></ul></li><li><span><a href="#Good-nutrition-impact" data-toc-modified-id="Good-nutrition-impact-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Good nutrition impact</a></span><ul class="toc-item"><li><span><a href="#High-nutrional-products" data-toc-modified-id="High-nutrional-products-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>High-nutrional products</a></span><ul class="toc-item"><li><span><a href="#Has-there-been-a-surge-in-high-graded-Products-in-the-UK-/-France-over-the-past-years?" data-toc-modified-id="Has-there-been-a-surge-in-high-graded-Products-in-the-UK-/-France-over-the-past-years?-3.2.1.1"><span class="toc-item-num">3.2.1.1&nbsp;&nbsp;</span>Has there been a surge in high graded Products in the UK / France over the past years?</a></span></li></ul></li><li><span><a href="#High-nutrional-products" data-toc-modified-id="High-nutrional-products-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>High-nutrional products</a></span><ul class="toc-item"><li><span><a href="#What-are-those-products-made-of?" data-toc-modified-id="What-are-those-products-made-of?-3.2.2.1"><span class="toc-item-num">3.2.2.1&nbsp;&nbsp;</span>What are those products made of?</a></span></li><li><span><a href="#Where-do-these-product-come-from-and-where-are-they-manufactured?" data-toc-modified-id="Where-do-these-product-come-from-and-where-are-they-manufactured?-3.2.2.2"><span class="toc-item-num">3.2.2.2&nbsp;&nbsp;</span>Where do these product come from and where are they manufactured?</a></span></li><li><span><a href="#Where-are-those-products-sold?" data-toc-modified-id="Where-are-those-products-sold?-3.2.2.3"><span class="toc-item-num">3.2.2.3&nbsp;&nbsp;</span>Where are those products sold?</a></span></li></ul></li><li><span><a href="#Carbon-footprint-of-nutrionally-high-graded-products" data-toc-modified-id="Carbon-footprint-of-nutrionally-high-graded-products-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Carbon footprint of nutrionally-high graded products</a></span><ul class="toc-item"><li><span><a href="#Is-there-a-general-correlation-between-high-carbon-footprint-and-price?" data-toc-modified-id="Is-there-a-general-correlation-between-high-carbon-footprint-and-price?-3.2.3.1"><span class="toc-item-num">3.2.3.1&nbsp;&nbsp;</span>Is there a general correlation between high carbon footprint and price?</a></span></li></ul></li></ul></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from scipy import stats
from datetime import datetime

import json
import pickle

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2
    
import libs.exploring as explore
import libs.visualising as visualize
import libs.cleansing as cleanse

# Set up plotly environment
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

## Import cleansed data

In [2]:
# Import data
open_food_facts_csv_file = "./data/openfoodfacts_clean.csv"

food_facts_pd = pd.read_csv(open_food_facts_csv_file,
                            delimiter="\t")

In [3]:
# Change column data types
food_facts_pd['carbon-footprint_100g'] = food_facts_pd['carbon-footprint_100g'].apply(pd.to_numeric, args=('coerce',))
food_facts_pd['energy_100g'] = food_facts_pd['energy_100g'].apply(pd.to_numeric, args=('coerce',))
food_facts_pd['price_per_100g'] = food_facts_pd['price_per_100g'].apply(pd.to_numeric, args=('coerce',))
food_facts_pd['created_datetime'] = food_facts_pd['created_datetime'].apply(pd.to_datetime, args=('coerce',))

In [4]:
food_facts_pd.head(5)

,Unnamed: 0,code,created_t,created_datetime,product_name,quantity,packaging,brands,categories_en,origins,...,ingredients_text,main_category,energy_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,price_per_100g,store_currency,manufacturing_place,purchase_places
0,0,0000000274722,1514659309,2017-12-30 18:41:49,Blanquette de Volaille et son Riz,NaN,"carton,plastique",Comme J’aime,"Meals,Meat-based products,Meals with meat,Poul...",Unknown,...,"Riz précuit 40,4 % (eau, riz, huile de colza, ...",Meats,450.0,NaN,0.0,0.0,NaN,NaN,France,France
1,1,0000000394710,1484497370,2017-01-15 16:22:50,Danoises à la cannelle roulées,1.150 kg,Frais,Kirkland Signature,"Sugary snacks,Biscuits and cakes,Pastries",France,...,"Ingrédients: Pâte (farine, eau, beurre, sucre,...",Sugary snacks,1520.0,NaN,NaN,NaN,NaN,NaN,France,France
2,2,0000001071894,1409411252,2014-08-30 15:07:32,Flute,NaN,"Paper,plastic film",Waitrose,"Plant-based foods and beverages,Plant-based fo...",Canada,...,Wheat,Plant-based,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Canada
3,3,0000001938067,1484501528,2017-01-15 17:32:08,Chaussons tressés aux pommes,1.200 kg,Frais,Kirkland Signature,"Sugary snacks,Biscuits and cakes,Pastries",France,...,"Ingrédients : Pâte (farine, margarines d'huile...",Sugary snacks,1090.0,NaN,9.0,9.0,NaN,NaN,United Kingdom,United Kingdom
4,4,0000004302544,1488464896,2017-03-02 14:28:16,Pain Burger Artisan,1.008 kg / 12 pain,"Frais,plastique",Kirkland Signature,boulange,Canada,...,"Ingrédients : Farine, eau, sel, levure, orge m...",Boulange,1160.0,NaN,1.0,1.0,NaN,NaN,Unknown,Canada


Additionally to the OpenFoodFact dataset, we obtained an extract of the Eaternity dataset hosted by the ETH Zurich, which contains 692 more products and their CO2 footprint. Unfortunately, these products are not contained in the OpenFoodFacts database, so we lack manufacturing and purchasing information as well as the OpenFoodFacts categories for this set.

In [5]:
# Import data
carbon_footprint_csv_file = "./data/carbon_footprint_categories.csv"

carbon_footprint_pd = pd.read_csv(carbon_footprint_csv_file, delimiter=",")
carbon_footprint_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 13 columns):
Unnamed: 0                      682 non-null int64
ID                              682 non-null int64
Title                           682 non-null object
Weight [gram/serving]           682 non-null int64
CO2-Value [gram CO2/serving]    682 non-null float64
CO2 rating                      682 non-null float64
FAT                             682 non-null float64
WATER                           682 non-null float64
ENERC                           682 non-null float64
PROT                            682 non-null float64
category                        682 non-null object
parent_category                 682 non-null object
category_en                     682 non-null object
dtypes: float64(6), int64(3), object(4)
memory usage: 69.3+ KB


In [6]:
carbon_footprint_pd.head(10)

,Unnamed: 0,ID,Title,Weight [gram/serving],CO2-Value [gram CO2/serving],CO2 rating,FAT,WATER,ENERC,PROT,category,parent_category,category_en
0,0,4300175162708,K Classic - Junger Gemüsemais,100,9.0,20.812,3.480252,52.999834,765.655520,8.601195,Gemüsekonserven,gemuese_pilze,vegetable mushrooms
1,1,4388840231829,ja! Gemüsemais,100,17.0,37.941,2.312597,35.218431,1070.401621,5.715417,Gemüsekonserven,gemuese_pilze,vegetable mushrooms
2,2,8690777653008,Sera Ajvar,100,44.0,61.076,11.526800,22.286400,1276.524000,12.058200,Gemüsaufstriche &amp; -salate,gemuese_pilze,vegetable mushrooms
3,3,4311527608225,Edeka Ackergold Mehligkochend,100,11.0,67.383,0.100000,78.700000,320.000000,2.000000,Kartoffeln,gemuese_pilze,vegetable mushrooms
4,4,7610632971826,Bio Kartoffeln festkochend,100,11.0,67.383,0.100000,78.700000,320.000000,2.000000,Kartoffeln,gemuese_pilze,vegetable mushrooms
5,5,4316268432429,Botato Kartoffel Wedges,100,20.0,71.343,4.032246,70.485923,512.666890,3.501101,Kartoffelbeilagen &amp; Pommes Frites,gemuese_pilze,vegetable mushrooms
6,6,4032600122055,Pfanni Kartoffel Püree besonders locker 3 x 80 g,100,12.0,73.344,0.099000,77.913000,316.800000,1.980000,Kartoffelbeilagen &amp; Pommes Frites,gemuese_pilze,vegetable mushrooms
7,7,4005009101204,Kartoffelpüree,100,14.0,82.441,0.197999,75.754800,310.801700,2.129955,Knödel &amp; Pürees,gemuese_pilze,vegetable mushrooms
8,8,4003880685257,Agrarfrost Germany Knusper Frites,100,22.0,84.098,5.488000,73.978000,500.600000,1.880000,Kartoffelbeilagen &amp; Pommes Frites,gemuese_pilze,vegetable mushrooms
9,9,4311596464074,Backofen Rösti Ecken - aus Qualitätskartoffeln,100,23.0,87.815,5.393437,72.587372,488.711426,1.947783,Kartoffelbeilagen &amp; Pommes Frites,gemuese_pilze,vegetable mushrooms


## Analyse data

Before we analyse the data, we have to some confesssions to make:

The data that we loaded into this notebook was already preprocessed in the "Open Food Facts - Cleanse data" notebook, that can be found in the same directory. In there we translated countries, labels, and categories and formatted and matched tags. However, we also droped more than 90% of the data set, because the data points were not complete for the purpose of our analysis.

OpenFoodFacts was initiated in France, and products sold in france are dominantly represented in this data set. Moreover, most of the products are sold in Europe or industrial nations, and we have no or only sparse data about the African, Asian, Australian, and South-american continent, which excludes the majority of the world population and especially the societes in Asia and Africa, that undergo the most decisive transformations at the moment.

Further, we only have qualitative data about the products, but no quantities that are produced and purchased world wide, hence we cannot provide a scale for all the insights that we gain throughout this notebook.

What we are trying to say is, that the data is under no circumstances representive to analyse the the research questions that we have posed. However, we will provide the methods to perform this analysis on this comprised dataset, and see what kind of insights we can already squeek out of it.

### Production / manufacture impact

#### Global distribution of global food producers

In [ ]:
countries_label = pd.read_csv("./data/country_lookup.csv")[['name', 'cca3']]     

##### Which are the dominant global food producers and manufacturers?

- From where are those products originating?

In [ ]:
visualize.plot_occurences_on_map(df=food_facts_pd, 
                                 column_key='origins',
                                 show_distances=False,
                                 title='Countries of origin for products')

Note that country in purple are  not assigned any value. 

- Where are those products manufactured?

In [ ]:
visualize.plot_occurences_on_map(df=food_facts_pd, 
                                 column_key='manufacturing_place',
                                 show_distances=False,
                                 title='Manufacturing countries of products')

- Where are those products bought?

In [ ]:
visualize.plot_occurences_on_map(df=food_facts_pd, 
                                 column_key='purchase_places',
                                 show_distances=False,
                                 title='Purchase countries of products')

In conclusion, we note that we mainly have data for "western" countries, with a <b> huge bias toward France</b>. We mostly lack information for country in Africa and the centre of Asia. Our dataset is thus clearly not a truthful representation of the world. We shall therefore restrict our analysis to the case of France, meaning purchases countries will be limited to the case of France. [This category was selected since it is the most furnished one.] 

This is carried out in the next cell. Note that <i> purchases_places </i> is only requested to contain 'France' as one of the entries in its list. There could thus be other countries still contained in the <i> purchases_places </i> column. 

In [ ]:
food_facts_pd['filter'] = food_facts_pd.purchase_places.apply(lambda l: explore.filter_france(l))
food_facts_pd = food_facts_pd[food_facts_pd['filter'] == 'France'].drop(columns=['filter'])

##### How is this distribution impacted when we consider neutral and large carbon footprint products? 

In [ ]:
# dataset carbon footprint coming from Eaternity
# This will be assess in future version of this project


#### Case study: Palm oil

##### Can we observe any trend in the number of products including this oil (assuming a strong dependence between date the product was added to the database and data the product was invented)?

In [ ]:
#extracting products with palm oil 
palm_oil_pd = food_facts_pd[food_facts_pd.ingredients_text.str.contains("palm").fillna(value=False)]

In [ ]:
print('{0:.2f} % of the products in the dataset contain palm oil'.format(palm_oil_pd.shape[0]/food_facts_pd.shape[0]*100))

In [ ]:
#palm_oil_pd.groupby('main_category')
palm_oil_pd['created_yyyy'] = palm_oil_pd["created_datetime"].dt.year

In [ ]:
#plotly.tools.set_credentials_file(username='ninatubau', api_key='z75HqORQkKdVL98Fi0tX')


In [ ]:
palm_oil_over_time = palm_oil_pd['created_yyyy'].value_counts()

In [ ]:
data = [go.Bar(x=palm_oil_over_time.index,
            y=palm_oil_over_time.values)]

layout = go.Layout(
    title='Usage of palm oil over time',
    xaxis=dict(
        title='time (years)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='number of products with palm oil',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='jupyter-basic_bar')

There is a clear tendancy of using palm oil in products from 2012 up to now. Also, we notice a large increase in the palm oil products these lasts years. 

##### Which country use palm oils for production?

In [ ]:
a = palm_oil_pd.origins.groupby(palm_oil_pd.origins).sum

In [ ]:
visualize.plot_column_composition(palm_oil_pd, 'manufacturing_place' )

In [ ]:
visualize.plot_occurences_on_map(df=food_facts_pd, 
                                 column_key='manufacturing_place',
                                 show_distances=True,
                                 title='Manufacturing countries of products')

### Good nutrition impact

In [ ]:
nutrition_fr = food_facts_pd[['product_name',
                              'created_datetime',
                              'nutrition-score-fr_100g', 
                              'product_name', 
                              'main_category', 
                              'origins', 
                              'purchase_places', 
                              'manufacturing_place',
                              'stores']
                            ]

nutrition_fr = nutrition_fr[nutrition_fr['nutrition-score-fr_100g'].notna()]
nutrition_over_time = nutrition_fr.sort_values(by = 'created_datetime')
nutrition_over_time['main_category'] = nutrition_over_time.main_category.fillna(value='Unknown')

Meanong of the nutrition score index can be found at https://world.openfoodfacts.org/nutriscore. The main facts are the following : 
- Products are marked according to the amount of nutrients they contain [per 100 g] and given a grade between A and E (A being obviously the best mark).

<img src="Images/nutriscore.png" height="540" width="336">

- If the product is solids, this is linked to a nutrition score as displayed the next table. This score itself is computed with two parts. The first one considers the energy, saturated fat, sugars and sodium. A high level in that category is considered unhealthy. The second part reflects the proportion of fruits, vegetables and nuts, fibers and proteins for which high levels are considered beneficial to the health.

<img src="Images/nutriscore_table.png" height="1000" width="900">



In [ ]:
#Assigning the grades
nutrition_over_time["nutrition_grade"] =\
                                    nutrition_over_time[['nutrition-score-fr_100g','main_category']].\
                                    apply(explore.assign_score, axis=1)


#### High-nutrional products

##### Has there been a surge in high graded Products in the UK / France over the past years?

In [ ]:
nutrition_over_time_reduced = explore.nutrition_grade(nutrition_over_time)

#### High-nutrional products

In [ ]:
visualize.make_grade_stacked_bar(nutrition_over_time_reduced, 'nutrition_grade', 'year', 'Count')                        

We observe that, as time passes, more products are being added with a nutritional grade, with a peak occurring during the years 2015-2016.

In [ ]:
visualize.make_grade_stacked_bar(nutrition_over_time_reduced, 'nutrition_grade', 'year', 'Percentage')

We observe that the percentage of prevalence of each grade has been mostly maintained during the last six years with a barely noticeable peak in 2013 for grade 'A'. 

In [ ]:
df = nutrition_over_time_reduced

In [ ]:
figure = {
    'data': [],
    'layout': {},
    'frames': [],
}

# fill in most of layout
figure['layout']['xaxis'] = {'title': 'Nutrtion Grade'}
figure['layout']['yaxis'] = {'title': '# Products'}
figure['layout']['hovermode'] = 'closest'

# add play and pause button
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

# define slider
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [ ]:
year = 2018
trace = go.Bar(
    x = df[df['year']==year].nutrition_grade.values,
    y = df[df['year']==year].Count.values,
    marker = dict(
        color = ["#008010", "#9ACD32","#FFD700", "#FF8C00", "#DB4832"]
        ),
    )

figure['data'].append(trace)

In [ ]:
for year in sorted(df.year.unique()):
    
    frame = {'data': [], 'name': str(year)}
    trace = go.Bar(
        x = df[df['year']==year].nutrition_grade.values,
        y = df[df['year']==year].Count.values,
        marker = dict(
            color = ["#008010", "#9ACD32","#FFD700", "#FF8C00", "#DB4832"]
            ),
    )
    
    frame['data'].append(trace)
    figure['frames'].append(frame)
    
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': int(year),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [ ]:
iplot(figure, filename='nutrition_grades')

##### What are those products made of?
What is the composition? Do they contain many additives?  Where are these products sold? 

This plot displays the most common categories in the list of product possessing a nutritional index

In [ ]:
visualize.make_content_stacked_bar(visualize.plot_grade_content(nutrition_over_time), 'keys', 'grade', 'Percentage')

Observe how good nutritional products are mostly (more than 50%) plant-based and how this category as well as carbs and canned food shrink when considering less beneficial food standards. This reduction is compensated by a sharp increase in prevalence of sugary snacks and a lesser increase of meat-based producs. Both seafood and dairy seem to concentre in, respectevely, the lower and higger part of the middle marks. 

##### Where do these product come from and where are they manufactured?

In [ ]:
visualize.plot_column_composition(nutrition_fr, 'purchase_places') 

This plot is of course highly bias towards france since the cut was perform on that column

In [ ]:
visualize.plot_column_composition(nutrition_fr, 'manufacturing_place') 

Naturally, most of the food consumed in France is manufactured there though approximately 30% is produced somewhere else. 

##### Where are those products sold?

In [ ]:
visualize.plot_column_composition(nutrition_fr, 'stores')

Side fact: [market share grocery stores in France](https://www.statista.com/statistics/535415/grocery-market-share-france/) (October 2017)
    <img src="Images/marketshare_stores_france.png" height="800" width="600">

#### Carbon footprint of nutrionally-high graded products

In this section we investigate the carbon footprint of different products and categories. Therefore, we investigate the OpenFoodFacts dataset. Common sense would suggest most nutritionally-high graded products are organic (plant, fruit, vegetables, …) and are therefore not manufactured, thus having a small footprint. Let's see what story the data has to tell...

But as before, we should be careful as the dataset is biased. So we begin with examing what kind of data is represented in the datasets.

We begin with extracting all the carbon footprint data.

In [ ]:
carbon_footprints = food_facts_pd[food_facts_pd['carbon-footprint_100g'].notna() & 
                                  food_facts_pd['carbon-footprint_100g']!=0]
carbon_footprints

First, we should sensibilize for the data that we are dealing with. Therefore we visualize the origin and composition of the products.

In [ ]:
visualize.plot_column_composition(df=carbon_footprints, column_str='manufacturing_place')

Let's if there is a correlation between country of origin, and hence transportation distance, and the carbon footprint. (No use because all products are from France or missing origin information).

In [ ]:
visualize.plot_column_composition(df=carbon_footprints, column_str='stores')

From the above plot, we can observe that the majority of the products are sold in france, and especially the stores in provided by the shops of

Most of the carbon footprint data was produced by the research group around ###, that claims that the processed data was ###.

In [ ]:
visualize.plot_column_composition(carbon_footprints, column_str='main_category')

We see that the main categories that we have carbon footprint data of are sugary snacks (mainly plain chocolets) and dairies. This is not surprising since they are made up of only few ingredients. 

In [ ]:
# Food calories over carbon-foot print
visualize.plot_cluster_by_tags(df=carbon_footprints,
                               plot2D_features = ["carbon-footprint_100g", "price_per_100g"],
                               cluster="main_category")

The above prices were found from online stores of Walmart, Monoprix, and Migros. It should be noted, that the dataset dominantly contains dairies and sweets with carbon footprint, so we hope to gain more insight of other products from the Carbon Footprint Eaternity dataset.


##### Is there a general correlation between high carbon footprint and price? 

Because the carbon footprint column of the OpenFoodFacts database contains a small amount of data, we will match OpenFoodFacts dataset with the sample given by Eaternity. The approach is the following :
    - Webscrap codeinfo website to get categories and parent categories of each product
    - Translate categories into english
    - Match categories of Open FoodFacts database with parent categories scraped from the website
    - Add the mean value of carbon footprint categories on Open Food Facts database
    

In [7]:
#for the 30 most common categories
food_facts_pd.main_category.value_counts().head(30)

Plant-based                    9654
Dairies                        5664
Sugary snacks                  4954
Meats                          4494
Beverages                      3713
Frozen foods                   1872
Groceries                      1411
Meals                          1344
Seafood                        1312
Carbs                          1063
Salty snacks                    868
Canned foods                    444
Sandwiches                      281
Desserts                        225
Sweeteners                      176
Fish And Meat And Eggs          174
Baby Foods                      142
Crêpes And Galettes             117
Dietary Supplements              93
Vinegars                         91
Aliments-D-Origine-Vegetale      77
Terrine                          75
Breakfasts                       54
Pies                             33
Pet Food                         32
Marzipan                         31
Food Additives                   31
Fats                        

In [8]:
print('Taking into account 30 most important categories represents {0}% of the data'.format(sum(food_facts_pd.main_category.value_counts()[:30])/len(food_facts_pd.main_category)*100))

Taking into account 30 most important categories represents 94.03561048286643% of the data


Dict done by hand to match main categories of the OpenFoodFact dataset with the one's found in the sample of Eternity. 

In [9]:
dict_categories = {
   "Plant-based": ['vegetable mushrooms','fruit berries','soy saitan meat substitute','Grain','nuts','seeds koerner','legumes','nut plant milk'],
    "Dairies": ['Milk, dairy products, eggs','Cheese','ice creme pudding'],
    "Sugary snacks":'confectionery',
    "Meats": 'Fruit and vegetables Meat',
    "Beverages":['Fruit and vegetable juices','Wine and Sparkling Wine','Syrup','cocoa drinking chocolate','Coffee','Mineral water','lemonades refreshment drinks','Milkshakes'],
    "Meals":'fast food whole ready meals',
    "Seafood":'fish sea fruits',
    "Groceries":['mayo ketchup mustard','finished sauces fix products','SPREADS'],
    "Carbs":['rice couscous quinoa co','Baked goods','breadsticks','muesli cereals'],
    "Fish And Meat And Eggs":'Eggs.',
    "Baby Foods":'shred',
    "Aliments-D-Origine-Vegetale":['vegetable mushrooms','fruit berries'],
    "Cooking Helpers": ['cooking ingredients spices','baking ingredients']
}


In [10]:
#computing carbon footprint mean for each parent category of Eaternity database
df_mean_carbon_eaternity = explore.carbon_mean_eaternity(carbon_footprint_pd)

In [11]:
carbon_footprint_pd.category_en.value_counts()

vegetable mushrooms             164
cooking ingredients spices       73
Fruit and vegetables Meat        49
Milk, dairy products, eggs       41
Baked goods                      35
confectionery                    32
breadsticks                      32
rice couscous quinoa co          22
Cheese                           21
shred                            17
fast food whole ready meals      17
muesli cereals                   17
mayo ketchup mustard             16
ice creme pudding                14
fruit berries                    14
fish sea fruits                  12
finished sauces fix products     12
Fruit and vegetable juices       12
baking ingredients               11
soy saitan meat substitute       10
Grain                             8
SPREADS                           7
seeds koerner                     7
nuts                              7
legumes                           7
nut plant milk                    6
Wine and Sparkling Wine           4
Syrup                       

In [13]:
df_mean_carbon_openff = explore.carbon_mean_openff(dict_categories,df_mean_carbon_eaternity)

In [14]:
df_mean_carbon_openff

,Plant-based,Dairies,Sugary snacks,Meats,Beverages,Meals,Seafood,Groceries,Carbs,Fish And Meat And Eggs,Baby Foods,Aliments-D-Origine-Vegetale,Cooking Helpers
0,96.048595,551.237408,174.5,622.714286,411.807778,230.941176,299.583333,1234.505265,1061.1444,219.5,82.0,2194.067109,2301.950048


In [ ]:
def campl

In [ ]:
completed_carbon_footprint = 

In [ ]:

#add values of carbon footprint on initial OpenFoodFacts database
food_facts_pd['correspond_cat']=None
for category in dict_categories : 
    food_facts_pd['carbon-footprint_100g'][food_facts_pd.main_category == category]=df_1[category][0]

In [ ]:
food_facts_pd['carbon-footprint_100g'].value_counts()

**Thanks for Reading !**